In [1]:
import numpy as np
import pandas as pd

from fwi_predict.constants import TZ_STRING

In [2]:
ara_raw = pd.read_excel("../data/raw/All ARA Data until March 31, 2024.xlsx", sheet_name='Sheet1')
var_dict = pd.read_excel("../data/raw/All ARA Data until March 31, 2024.xlsx", sheet_name='Sheet2')

The unique identified should be the pond ID and sample datetime.

In [3]:
ara_raw['Group'].value_counts()

Group
ARA V1 (2-weekly)                    2643
ARA V1.2 (regular-monthly)           1889
Regular ARA pond                     1117
Feed Holiday Efficacy Test (2023)     563
Focus Group 4                         215
ARA V1.2 (in-depth-weekly)            209
Focus Group 3                         190
Test A (Farmer Measurements)           84
Test A (Water Sample Analysis)         43
Focus Group 2                          24
Test B                                 16
Focus Group 1                           4
Name: count, dtype: int64

In [4]:
# Helpers
def is_str(x) -> bool:
    return isinstance(x, str)

## Data sanity checks

In [5]:
column_map = {
    'Date of Data Collection': 'date',
    'Pond ID': 'pond_id',
    'Measurement Type': 'time_of_day',
    'Group': 'group',
    'Pond Type': 'treatment_group',
    'Time (sample or data collection)': 'sample_time',
    'Name': 'name', # Is this the owner?
    'Which meter are you using?': 'measure_instrument',
    'Follow-up': 'follow_up',
    'Weather': 'weather',
    'Dissolved Oxygen (mg/L)': 'do_mg_per_L',
    'pH': 'ph',
    'Turbidity (cm)': 'turbidity_cm',
    'Ammonia (mg/L)': 'ammonia_mg_per_L',
    'Water quality in the *required* range': 'in_range',
    'Parameter(s) out of range': 'params_out_of_range',
    'Temperature (°C)': 'temperature_celsius',
    'Conductivity (ms)': 'conductivity_ms',
    'TDS (ppt)': 'tds_ppt',
    'Water Color': 'water_color',
    'Corrective actions requested': 'corrective_actions_requested',
    'Amount requested': 'amount_requested',
    'Corrective actions implementation': 'corrective_actions_implementation',
    'Corrective actions implementation date': 'corrective_actions_implementation_date',
    'Corrective actions taken': 'corrective_actions_taken',
    'Non-prescribed actions taken': 'non_prescribed_actions_taken',
    'If the farmer didn\'t apply corrective actions, what were the reasons?': 'no_corrective_reasons',
    'Improvement of *targeted* water quality parameters': 'targeted_params_improvement',
    'Notes (corrective actions)': 'corrective_action_notes',
    'Individuals air gulping': 'individuals_air_gulping',
    'Individuals tail splashing': 'individuals_tail_splashing',
    'Dead fish': 'dead_fish',
    'Notes (mortalities)': 'mortalities_notes',
    'Feed amount (kg)': 'feed_amount_kg',
    'Did we help the fish?': 'did_we_help_the_fish',
    'Stocking density (fish per acre)': 'fish_per_acre',
    'Species': 'species',
    'Weight': 'weight', # Need to find units
    'Notes (additional info)': 'additional_info',
    'Any pictures you want to share?': 'pictures',
    'Data tool Sr. No.': 'serial_no_data_tool',
    'Winkler\'s Method Used for DO': 'winklers_method',
    'Feed type': 'feed_type',
    'Days without feed since the last measurement': 'days_without_feed_since_last_measurement',
    'Chlorophyll-a': 'chl-a',
    'Phycocyanin': 'phycocyanin',
    'Plankton Sample Analysis Date': 'plankton_sample_analysis_date',
    'Total n° of cells / 1L': 'cells_per_L',
    'Submission ID': 'submission_id', # Probably refer to plankton sample
    'Dead fish found by the FARMER since the last visit': 'dead_fish_since_last_visit_farmer_report',
    'Dead fish found by YOU today': 'dead_fish_found_fwi',
    'How many locations?': 'num_locations', # Is this the number of sampling locations?
    'Prescribed collection date': 'prescribed_collection_date',
    'Reason for late or no collection (if any)': 'no_or_late_collection_reason',
    'Did the farmer collect the measurement?': 'farmer_collected_measurement',
    'Turbidity (farmer measurement)': 'turbidity_farmer',
    'Temperature (farmer measurement)': 'temperature_farmer',
    'pH (farmer measurement)': 'ph_farmer',
    'Feed given today': 'feed_given_today',
    'Time (sample analysis)': 'sample_analysis_time',
    '1. Dissolved Oxygen (mg/L)': 'do_mg_per_L_1',
    '2. Dissolved Oxygen (mg/L)': 'do_mg_per_L_2',
    '3. Dissolved Oxygen (mg/L)': 'do_mg_per_L_3',
    '1. pH': 'ph_1',
    '2. pH': 'ph_2',
    '3. pH': 'ph_3',
    '1. Temperature (in °C)': 'temperature_celsius_1',
    '2. Temperature (in °C)': 'temperature_celsius_2',
    '3. Temperature (in °C)': 'temperature_celsius_3',
    'Light bottle DO (NPP)': 'light_bottle_do_npp',
    'Dark bottle DO (R)': 'dark_bottle_do_R',
    'Outcomes of corrective actions': 'corrective_actions_outcome',
    'Salinity (ppt)': 'salinity_ppt',
    'Feeding': 'feeding',
    'Air gulping': 'air_gulping',
    'Primary Productivity GPP (mg/L)': 'primary_productivity_gpp_mg_per_L',
    'Alkalinity': 'alkalinity',
    'How did the farmer find implementing the corrective actions?': 'corrective_actions_farmer_ease',
    'Tail splashing': 'tail_splashing',
    'How many fish did we help?': 'fish_helped',
    'How did we help the fish?': 'fish_help_method',
    'Readings communicated today': 'readings_communicated_today',
    'Actions taken': 'actions_taken',
    'Details': 'details',
    'Wind': 'wind',
    'Disease outbreak': 'disease_outbreak',
    'Lice infestation': 'lice_infestation',
    'Vegetation (1+ cm into the water)': 'vegetation_in_water'
} 

In [6]:
ara = ara_raw.rename(columns=column_map)
assert(ara.columns.isin(column_map.values()).all()) # Assert column names standardized

In [10]:
# Clean sample times
ara['sample_time'].apply(type).value_counts() 
str_formatted = ara['sample_time'].apply(is_str)
ara.loc[str_formatted, 'sample_time'] # Find string formatted times
ara.loc[str_formatted, 'sample_time'] = ara.loc[str_formatted, 'sample_time'] + ':00' # Add seconds
ara.loc[~str_formatted  & ara['sample_time'].notna(), 'sample_time'] = \
    ara.loc[~str_formatted & ara['sample_time'].notna(), 'sample_time'].apply(lambda x: x.strftime("%H:%M:%S"))

In [11]:
ara['sample_dt'] = pd.to_datetime(ara['date'].dt.strftime("%Y-%m-%d") + ' ' + ara['sample_time'], errors='coerce')
ara['sample_dt'] = ara['sample_dt'].dt.tz_localize(TZ_STRING) # Add timezone
ara = ara.drop(columns=['date', 'sample_time'])

In [12]:
yes_no_map = {
    'Yes': True,
    'No': False
}

In [13]:
ara['follow_up'] = ara['follow_up'].map(yes_no_map)
ara['follow_up'].value_counts()

follow_up
False    5288
True      768
Name: count, dtype: int64

In [14]:
ara['turbidity_cm'].apply(type).value_counts(dropna=False)
str_formatted = ara['turbidity_cm'].apply(is_str)
ara.loc[str_formatted, 'turbidity_cm'] = np.nan
ara['turbidity_cm'] = ara['turbidity_cm'].astype(float)

In [15]:
ara['ammonia_mg_per_L'].apply(type).value_counts(dropna=False)
str_formatted = ara['ammonia_mg_per_L'].apply(is_str)
ara.loc[str_formatted, 'ammonia_mg_per_L'] = np.nan
ara['ammonia_mg_per_L'] = ara['ammonia_mg_per_L'].astype(float)

In [16]:
ara['in_range'] = ara['in_range'].map(yes_no_map)

In [17]:
ara['params_out_of_range'].value_counts()

params_out_of_range
Dissolved oxygen                                       852
pH                                                     245
Dissolved oxygen\npH                                   166
Dissolved oxygen\nTurbidity                             52
Turbidity                                               44
Dissolved oxygen\nAmmonia                               31
Dissolved oxygen\npH\nAmmonia                           27
Ph                                                      24
Ammonia                                                 20
Dissolved Oxygen                                        18
pH\nAmmonia                                             17
Dissolved oxygen\npH\nAmmonia\nPrimary productivity     12
pH\nAmmonia\nPrimary productivity                        7
Dissolved oxygen\nAmmonia\nPrimary productivity          7
Dissolved Oxygen, Ph                                     7
pH\nDissolved oxygen                                     7
Dissolved oxygen\nPrimary productivi

Later create own OOR variables and drop this var

In [18]:
ara['temperature_celsius'].apply(type).value_counts()
str_formatted = ara['temperature_celsius'].apply(is_str)
ara.loc[str_formatted, 'temperature_celsius'] = np.nan # Str formatted temperature is illegible so setting to nan.
ara['temperature_celsius'] = ara['temperature_celsius'].astype(float)

In [19]:
ara['conductivity_ms'].value_counts()
ara['conductivity_ms'].apply(type).value_counts()
str_formatted = ara['conductivity_ms'].apply(is_str)
ara.loc[str_formatted, 'conductivity_ms'] = ara.loc[str_formatted, 'conductivity_ms'] \
    .str.replace('o', '0') \
    .str.replace('`', '') \
    .str.replace(' ', '') \
    .str.replace('\'', '.')
ara['conductivity_ms'] = ara['conductivity_ms'].astype(float)

In [20]:
ara['tds_ppt'].apply(type).value_counts()
str_formatted = ara['tds_ppt'].apply(is_str)
ara.loc[str_formatted, 'tds_ppt'] = ara.loc[str_formatted, 'tds_ppt'] \
    .str.replace('`', '') \
    .str.replace('l', '1')
ara['tds_ppt'] = ara['tds_ppt'].astype(float)

In [21]:
ara['water_color'].value_counts(dropna=False) # Not important to clean

water_color
Light green                                2264
Dark green                                 2143
Light brown                                 822
Saturated green                             501
NaN                                         387
Transparent                                 282
Green                                       235
Brown                                       107
Brownish                                     68
Dark brown                                   42
White / Transparent                          41
White/transparent                            33
Yellowish brown                              22
brown                                        21
White                                        17
green                                        15
Rusty brown                                   9
Light Green                                   5
dark green                                    4
Blue-green                                    3
Greenish                    

In [22]:
ara['feed_amount_kg'].value_counts() # Complicated so won't try to convert to numeric
ara['did_we_help_the_fish'] = ara['did_we_help_the_fish'].map(yes_no_map)

In [23]:
ara['fish_per_acre'].value_counts()
ara['fish_per_acre'].apply(type).value_counts()
str_formatted = ara['fish_per_acre'].apply(is_str)
ara.loc[str_formatted, 'fish_per_acre'] = np.nan
ara['fish_per_acre'] = ara['fish_per_acre'].astype(float)

In [24]:
# Ask about units
ara['weight'].value_counts()
ara['weight'].apply(type).value_counts()
str_formatted = ara['weight'].apply(is_str)
ara.loc[str_formatted, 'weight'] # One gives units in kg so only clean after askings

1610    gra
3594    2KG
Name: weight, dtype: object

In [25]:
ara['winklers_method'] = ara['winklers_method'].map(yes_no_map)

In [26]:
ara['days_without_feed_since_last_measurement'].apply(type).value_counts()
str_formatted = ara['days_without_feed_since_last_measurement'].apply(is_str)
ara.loc[str_formatted, 'days_without_feed_since_last_measurement']
ara.loc[ara['days_without_feed_since_last_measurement'].str.contains('second day') &
        ara['days_without_feed_since_last_measurement'].notna(), # Fix later
        'days_without_feed_since_last_measurement'] = 2
ara.loc[ara['days_without_feed_since_last_measurement'].str.contains('once') &
        ara['days_without_feed_since_last_measurement'].notna(),
        'days_without_feed_since_last_measurement'] = np.nan # Set 'Weekly once' to null for now
ara['days_without_feed_since_last_measurement'] = ara['days_without_feed_since_last_measurement'].astype(float)    

In [27]:
ara['dead_fish_since_last_visit_farmer_report'].value_counts()
ara['dead_fish_since_last_visit_farmer_report'].apply(type).value_counts()
str_formatted = ara['dead_fish_since_last_visit_farmer_report'].apply(is_str)
ara.loc[str_formatted, 'dead_fish_since_last_visit_farmer_report'] = 0
ara['dead_fish_since_last_visit_farmer_report'] = ara['dead_fish_since_last_visit_farmer_report'].astype(float)

In [28]:
ara.loc[ara['num_locations'].str.contains('1') & ara['num_locations'].notna(), 'num_locations'] = 1
ara['num_locations'] = ara['num_locations'].astype(float)

In [29]:
ara['prescribed_collection_date'].value_counts()
str_formatted = ara['prescribed_collection_date'].apply(is_str)
ara.loc[str_formatted, 'prescribed_collection_date'] = ara.loc[str_formatted, 'prescribed_collection_date'] \
    .str.extract('(\\d{2}/\\d{2}/\\d{4})') \
    .squeeze() \
    .pipe(pd.to_datetime)
ara['prescribed_collection_date'] = ara['prescribed_collection_date'].astype('datetime64[ns]')

In [30]:
ara['feed_given_today'] = ara['feed_given_today'].map(yes_no_map)

In [31]:
ara['sample_analysis_time'].value_counts()
ara['sample_analysis_time'].apply(type).value_counts()

sample_analysis_time
<class 'float'>            6984
<class 'datetime.time'>      55
Name: count, dtype: int64

In [32]:
ara['do_mg_per_L_1'].apply(type).value_counts()
str_formatted = ara['do_mg_per_L_1'].apply(is_str)
ara.loc[str_formatted, 'do_mg_per_L_1'] = ara.loc[str_formatted, 'do_mg_per_L_1'] \
    .str.replace(' ', '') \
    .str.replace('O', '0')
ara['do_mg_per_L_1'] = ara['do_mg_per_L_1'].astype(float)

ara['temperature_celsius_1'].apply(type).value_counts()
str_formatted = ara['temperature_celsius_1'].apply(is_str)
ara.loc[str_formatted, 'temperature_celsius_1'] = ara.loc[str_formatted, 'temperature_celsius_1'] \
    .str.replace('..', '.') \
    .str.replace(' ', '')
ara['temperature_celsius_1'] = ara['temperature_celsius_1'].astype(float)

In [33]:
ara['light_bottle_do_npp'].value_counts()
ara['light_bottle_do_npp'].apply(type).value_counts()
str_formatted = ara['light_bottle_do_npp'].apply(is_str)
ara.loc[str_formatted, 'light_bottle_do_npp'] = ara.loc[str_formatted, 'light_bottle_do_npp'] \
    .str.replace('`', '') \
    .replace('NC', np.nan)
ara['light_bottle_do_npp'] = ara['light_bottle_do_npp'].astype(float)

In [34]:
ara['dark_bottle_do_R'].apply(type).value_counts()
str_formatted = ara['dark_bottle_do_R'].apply(is_str)
ara.loc[str_formatted, 'dark_bottle_do_R'] = ara.loc[str_formatted, 'dark_bottle_do_R'] \
    .str.replace('o', '0') \
    .replace('NC', np.nan)
ara['dark_bottle_do_R'] = ara['dark_bottle_do_R'].astype(float)

In [35]:
ara['salinity_ppt'].value_counts()
ara['salinity_ppt'].apply(type).value_counts()
str_formatted = ara['salinity_ppt'].apply(is_str)
ara.loc[str_formatted, 'salinity_ppt'] = ara.loc[str_formatted, 'salinity_ppt'] \
    .str.replace(' ', '') \
    .str.replace(',', '.')
ara['salinity_ppt'] = ara['salinity_ppt'].astype(float)

In [36]:
ara['primary_productivity_gpp_mg_per_L'].value_counts()
ara['primary_productivity_gpp_mg_per_L'].apply(type).value_counts()
str_formatted = ara['primary_productivity_gpp_mg_per_L'].apply(is_str)
ara.loc[str_formatted, 'primary_productivity_gpp_mg_per_L'] = np.nan # All unintelligible
ara['primary_productivity_gpp_mg_per_L'] = ara['primary_productivity_gpp_mg_per_L'].astype(float)

In [37]:
ara['readings_communicated_today'] = ara['readings_communicated_today'].map(yes_no_map)

In [38]:
ara['disease_outbreak'] = ara['disease_outbreak'].map(yes_no_map)

In [39]:
ara['disease_outbreak'].value_counts(dropna=False)

disease_outbreak
NaN      4988
False    2038
True       13
Name: count, dtype: int64

In [40]:
ara['lice_infestation'] = ara['lice_infestation'].map(yes_no_map)

In [41]:
ara['vegetation_in_water'] = ara['vegetation_in_water'].map(yes_no_map)

Deduplicate

In [91]:
def resolve_duplicates(
    df, id_cols, string_delimiter="; ", mark_column="had_duplicates"
):
    """
    Resolve duplicates in a DataFrame, marking which rows had duplicates and resolving conflicts.

    Args:
        df (pd.DataFrame): The input DataFrame containing duplicates.
        id_cols (list): List of columns defining the unique ID.
        string_delimiter (str, optional): Delimiter to concatenate strings.
        mark_column (str, optional): Name of the column to indicate duplicates.

    Returns:
        pd.DataFrame: A DataFrame with duplicates resolved and marked.
    """
    def resolve_group(group):
        resolved = {}
        for col in group.columns:
            if col in id_cols:
                # Keep ID columns as is
                resolved[col] = group[col].iloc[0]
            else:
                if pd.api.types.is_numeric_dtype(group[col]):
                    # Resolve numeric columns by mean, ignoring NaNs
                    resolved[col] = group[col].mean(skipna=True)
                elif pd.api.types.is_string_dtype(group[col]):
                    # Resolve string columns by concatenating unique values
                    unique_strings = group[col].dropna().unique()
                    resolved[col] = string_delimiter.join(unique_strings)
                else:
                    # Cast other types to strings and concatenate unique values
                    unique_values = group[col].dropna().astype(str).unique()
                    resolved[col] = string_delimiter.join(unique_values)
        
        # Mark duplicates if the group contains more than one row
        resolved[mark_column] = len(group) > 1
        return pd.Series(resolved)

    # Apply resolution to each group
    resolved_df = (
        df.groupby(id_cols)
        .apply(resolve_group, include_groups=False)
        .reset_index()
    )

    return resolved_df


def find_conflicting_duplicates(df, id_cols):
    """
    Identify duplicates with conflicting values across other columns.

    Args:
        df (pd.DataFrame): The input DataFrame containing potential duplicates.
        id_cols (list): List of columns defining the unique ID.

    Returns:
        pd.DataFrame: A DataFrame of rows with conflicting values.
    """
    # Helper function to check for conflicts within each group
    def has_conflicts(group):
        for col in group.columns.difference(id_cols):
            unique_values = group[col].dropna().unique()
            if len(unique_values) > 1:
                return True
        return False

    # Group by ID columns and filter conflicting groups
    conflicting_groups = df.groupby(id_cols).filter(has_conflicts)
    return conflicting_groups

def find_conflicting_columns(df, id_cols):
    """
    Identify duplicate groups and the columns with conflicts.

    Args:
        df (pd.DataFrame): Input DataFrame containing duplicates.
        id_cols (list): List of columns defining the unique ID.

    Returns:
        pd.DataFrame: A DataFrame containing duplicate groups with conflicting columns identified.
    """
    def check_conflicts(group):
        # Find columns where there are conflicting values
        conflicting_columns = [
            col for col in group.columns if col not in id_cols and group[col].nunique(dropna=True) > 1
        ]
        result = group.copy()
        result["conflicting_columns"] = ", ".join(conflicting_columns) if conflicting_columns else None
        return result

    # Identify duplicate groups and check for conflicts
    duplicate_groups = df[df.duplicated(subset=id_cols, keep=False)]
    conflicting_groups = duplicate_groups.groupby(id_cols, group_keys=False).apply(check_conflicts)
    conflicting_groups = conflicting_groups.reset_index(drop=True)

    return conflicting_groups

In [45]:
conflicting = find_conflicting_duplicates(ara, ['pond_id', 'sample_dt'])

In [90]:
deduped = resolve_duplicates(ara, ['pond_id', 'sample_dt'])

In [55]:
conflicting_cols = find_conflicting_columns(ara, ['pond_id', 'sample_dt'])

C:\Users\squaade\AppData\Local\Temp\ipykernel_23156\845596098.py:94: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  conflicting_groups = duplicate_groups.groupby(id_cols, group_keys=False).apply(check_conflicts)


In [68]:
conflicting_cols[conflicting_cols['conflicting_columns'].str.contains('in_range') &
                 conflicting_cols['conflicting_columns'].notna()]

,pond_id,time_of_day,group,treatment_group,name,measure_instrument,follow_up,weather,do_mg_per_L,ph,...,fish_help_method,readings_communicated_today,actions_taken,details,wind,disease_outbreak,lice_infestation,vegetation_in_water,sample_dt,conflicting_columns
578,WG-AJU1,Morning,Test A (Farmer Measurements),Treatment,Nagaraju P.,NaN,False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Light breeze (1-5 knots),False,False,NaN,2022-12-08 07:35:00+05:30,"group, in_range, submission_id"
579,WG-AJU1,Morning,Test A (Water Sample Analysis),Treatment,Nagaraju P.,NaN,False,NaN,1.1,7.72,...,NaN,NaN,NaN,NaN,Light breeze (1-5 knots),False,False,NaN,2022-12-08 07:35:00+05:30,"group, in_range, submission_id"


In [66]:
import itertools

conflict_colnames = conflicting_cols['conflicting_columns'].str.split(", ").tolist()
conflict_colnames = set(itertools.chain.from_iterable([l for l in conflict_colnames if isinstance(l, list)]))
conflict_colnames

{'corrective_actions_taken',
 'group',
 'in_range',
 'lice_infestation',
 'params_out_of_range',
 'submission_id',
 'vegetation_in_water',
 'wind'}

For now, these are all variables that you trust. This means creating your own in_range measures and maybe not using the group variable. We will see if that's possible.

For now just proceed and save.

In [69]:
ara_deduped = resolve_duplicates(ara, ['pond_id', 'sample_dt'])

C:\Users\squaade\AppData\Local\Temp\ipykernel_23156\845596098.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(resolve_group)


In [70]:
front_cols = ['pond_id', 'group', 'treatment_group', 'sample_dt', 'time_of_day']
ara_deduped = ara_deduped[front_cols + [col for col in ara.columns if col not in front_cols]]

Get duplicates to send to Jennifer

In [77]:
duplicates = ara[ara.duplicated(['pond_id', 'sample_dt'], keep=False)] \
  .sort_values(['pond_id', 'sample_dt']) \
  .reset_index() \
  .pipe(find_conflicting_columns, id_cols=['pond_id', 'sample_dt'])

C:\Users\squaade\AppData\Local\Temp\ipykernel_23156\845596098.py:94: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  conflicting_groups = duplicate_groups.groupby(id_cols, group_keys=False).apply(check_conflicts)


In [84]:
front_cols = ['index', 'conflicting_columns', 'pond_id', 'group', 'treatment_group', 'sample_dt', 'time_of_day']
duplicates = duplicates[front_cols + [col for col in duplicates.columns if col not in front_cols]]
duplicates['sample_dt'] = duplicates['sample_dt'] \
  .dt.tz_localize(None) \
  .dt.strftime("%Y-%m-%d %H:%M:%S") # To save as excel

duplicates.to_excel("../data/clean/ara_data_josiah_duplicates.xlsx")